In [1]:
from pytorch_unet import *
from torch import nn
import pandas as pd
from timerit import Timerit
from IPython.display import display, HTML

In [2]:
print(torch.__version__)

1.9.0+cu102


In [3]:
model = UNet(128,2).eval()

x = torch.randn((2,1,128,128))
pred = model(x)

/home/nick/anaconda3/lib/python3.7/site-packages/torch/nn/modules/conv.py:440: UserWarning: Using padding='same' with even kernel lengths and odd dilation may require a zero-padded copy of the input be created (Triggered internally at  /pytorch/aten/src/ATen/native/Convolution.cpp:660.)
  self.padding, self.dilation, self.groups)


In [4]:
metric_labels = [
    "Frames",
    "# Samples",
    "First PT (s)",
    "Avg PT (s)",
    "Min PT (s)",
    "Max PT (s)",
    "Std. Dev. (s)",
]

device = " "
if torch.cuda.is_available():
    device = torch.cuda.get_device_name(torch.cuda.current_device())
else:
    device = 'cpu'
    
print('Using device:', device)

results_df = pd.DataFrame(columns = metric_labels)
frames = [2, 10, 20, 30, 40, 50]
for l in range(len(frames)):
    model = UNet(128,2).eval()
    t = Timerit(num = 100, verbose=0)
    
    for timer in t:
        x = torch.randn((frames[l],1,128,128))
        with timer:
            pred = model(x)
    
    mf_times = t.times
    
    results_df.loc[l] = [
        frames[l],
        100,
        mf_times[0],
        int((np.mean(mf_times)*100000)) / 100000,
        np.min(mf_times),
        np.max(mf_times),
        int((np.std(mf_times)*100000)) / 100000
    ]

print('Pytorch Multi-Frame Batch Prediction Timings')
display(results_df)

Using device: GeForce RTX 2060 SUPER
Pytorch Multi-Frame Batch Prediction Timings


,Frames,# Samples,First PT (s),Avg PT (s),Min PT (s),Max PT (s),Std. Dev. (s)
0,2.0,100.0,0.010037,0.00860,0.007926,0.010307,0.00056
1,10.0,100.0,0.035724,0.02642,0.022504,0.039696,0.00446
2,20.0,100.0,0.053974,0.04774,0.044738,0.066648,0.00289
3,30.0,100.0,0.074226,0.07259,0.065999,0.099331,0.00681
4,40.0,100.0,0.097134,0.10662,0.091783,0.150557,0.01734
5,50.0,100.0,0.139550,0.12446,0.115261,0.176546,0.01020
